Hello Everyone. The following code is a work on image-processing which aims to make understanding and working with DICOM data-set simple for any user. As you will go ahead you will find that minute details such as why to convert to HU values ,what DICOM data-set looks like, what is there in DICOM's Pixel Data attribute, what does those value represent corresponding to CT Scan etc. are emphasized for making readers know the background flow of steps. This is  basically our the first submission on Kaggle platform. Thanks to you guys for bringing up such a good problem definition which made us ponder on learning new concepts and techniques in the field of computer aided image processing.

In [ ]:
# All imports and data path

import os
import pandas as pds
import dicom
import numpy as npy
import matplotlib.pyplot as ppt
import scipy.ndimage

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection



main_dir ='../input/sample_images/' # Input DIR which contains data of all the patients
patients_data = os.listdir(main_dir) # List of folders of all patients which conatins scan data
#label_set = pds.read_csv('../input/stage1_labels.csv', index_col=0)

#label_set.head() # or alternatively one can use "print(label_set)" if one wishes to see entire file for all the patients  

In [ ]:
# 'Voxels' is a term used more often in the entire code from here onwards and it represents each slice of the thoracic volume

# Reading all voxels of each patient in a list named voxels, this list will be manipulated here onwards

def voxel_read(list_Patients_Data):
    
    # thanks to --> https://www.kaggle.com/sentdex/data-science-bowl-2017/first-pass-through-data-w-3d-convnet for precise and dynamic idea of reading

    for patient_data in patients_data [:1]:
        #label = label_set.get_value(patient_data, 'cancer')
        path_each_pat = main_dir + patient_data

        
    
        voxels= [dicom.read_file(path_each_pat + '/' + img) for img in os.listdir(path_each_pat)]
        voxels.sort(key = lambda x:int(x.InstanceNumber)) # Instance Number is an attribute of DICOM, it identifies images in series (Image ID)
                                                          #or
                                                          #voxels.sort(key=lambda x:int(x.ImagePositionPatient[2]))
    
        #Calculating the attribute SliceThickness and adding it in the list voxels

        slice_thickness= npy.abs(voxels[0].ImagePositionPatient[2]-voxels[1].ImagePositionPatient[2]) #one of the important attributes which will used in later processing
                                                                                                            
        print("Thickness of Each slice: %f " %slice_thickness)
        
        for img in voxels:
            img.SliceThickness=slice_thickness
            
        print("No. of VOXELS : ", len(voxels),"\t")     
            
    
        return npy.array(list(voxels))
    
v=voxel_read(patients_data) 
print("\nDICOM file of a slice:\n\n")  # This gives an actual DICOM file of one of the scanned image.
print(v[10])

In [ ]:
#Gray-scale plot of some of the slices of a patient

"""X=ppt.figure(figsize=(20,20))


for i,image in enumerate (v [50:74]):
    X.add_subplot(6,4,i+1)
    img1= image.pixel_array
    ppt.imshow(img1,cmap=ppt.cm.gray)
ppt.show()"""

In [ ]:
# understanding and visualizing values contained in DICOM data which corresponds to pixel data

# NOTE: for purpose of efficient storage values stored in Pixel Data are Attenuation Coefficient
# Lets visualize these data

ppt.imshow(v[100].pixel_array,cmap=ppt.cm.gray)
ppt.show()
print(v[100].pixel_array[10]) # pixel_array is a numpy function to read data in to nD-array

In [ ]:
#HounsField Conversion fun

def attValues_to_HU(voxels):

    #Getting a join sequence of all the voxels(slices) in a patient 
    #NOTE: Each voxel which is a list contains atteunation coefficent of scanned thoraic portion respectively
    Array_of_voxels = npy.stack([v.pixel_array for v in voxels])
    #print(Array_of_voxels)  #this print stt. is only for once to check the output
    
    #Attenuation values are effficent for storage on disk, but in memory compution requires HU values
    #HU ranges from nearly -1000 to some thousand on +ve scale, 
    #Thus conversion to type interger 16 bit is necessary for attenutation coefficient values
    Array_of_voxels=Array_of_voxels.astype(npy.int16)
    # print(Array_of_voxels[10][10])
    
    #omitting unwanted -2000 att. coeff. for box shaped images
    Array_of_voxels[Array_of_voxels==-2000]=0
    #print(Array_of_voxels[10][10])  #this print stt. is only for once to check the output
    
    
    #Any voxel or sclice can be chosen for a patient, they will contain same SLOPE and INTERCEPT VALUE
    Slope=voxels[0].RescaleSlope
    Intercept=voxels[0].RescaleIntercept
    
    print("\n\nRESCALE SLOPE \t RESCALE INTERCEPT: ",Slope,"\t",Intercept)

    if Slope != 1: #which is generally 1 in most of the case 
        Array_of_voxels= Slope * Array_of_voxels.astype(npy.float64)
        Array_of_voxels= Array_of_voxels.astype(npy.int16)
    

    Array_of_voxels= npy.int16(Intercept) + Array_of_voxels   
    #print(Array_of_voxels[100])

    return npy.array(Array_of_voxels, dtype=npy.int16)


Voxel_HU_List=attValues_to_HU(v)

print("\n\n Array contaning HU values of PIXEL DATA: \n\n")
print(Voxel_HU_List)
print("\n\n Single row values of a VOXEL: \n\n")
print(Voxel_HU_List[100][10])

#Histogram Plotting of one patient

list_of_voxels= Voxel_HU_List

print("\n\n HISTOGRAM PLOT OF HU VALUES OF A PATIENT: \n\n")

ppt.hist(list_of_voxels.flatten(), bins=50, color='crimson',histtype='bar')
ppt.xlabel("Hounsfield Units (HU)")
ppt.ylabel("Frequency")
ppt.show()

In [ ]:
#Experimental code for Sampling

Pixel_Spacing = map(float, ([v[0].SliceThickness] + v[0].PixelSpacing))
Pixel_Spacing = npy.array(list(Pixel_Spacing))
print("\n\nPixel Spacing values as --> [SLICE_THICKNESS\t PIXEL_SPACING_x\t PIXEL_SPACING_y]  :\n")
print(Pixel_Spacing)
print("\n\nShape of the scanned thorax --> (LENGHT, WIDTH, HEIGHT) in terms of pixels: \n")
print(Voxel_HU_List.shape)

In [ ]:
# RESAMPLING (basically resizing the no of pixels keeping dimensional measurement same)

def Resample(slices, images, new_pixel_spacing=[1,1,1]):
    
    Pixel_Spacing = map(float, ([slices[0].SliceThickness] + slices[0].PixelSpacing))
    Pixel_Spacing = npy.array(list(Pixel_Spacing))
    
     # some lines from: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
         # The mathematical computation is same all other way, which very precisely is described below   
    
    resize_factor = Pixel_Spacing / new_pixel_spacing
    new_real_shape = images.shape * resize_factor
    new_shape = npy.round(new_real_shape)
    real_resize_factor = new_shape / images.shape
    new_pixel_spacing = Pixel_Spacing / real_resize_factor

    Array_of_voxels = scipy.ndimage.interpolation.zoom(images, real_resize_factor)
    
    return Array_of_voxels, new_pixel_spacing

img2,res=Resample(v,Voxel_HU_List,[1,1,1])
print("\n\n New SHAPE , NEW SPACING:", img2.shape,res)

In [ ]:
# Lets get the Pixel values standardized or basically normalized to a standarad
# This step is crucial as it used for feature scaling which later will help machince learning algos. like CLASSIFERS for better categorization
#There are two of such methods namely: Rescaling and Standardization , here later is preferred as we deal with muti-D array values.
# For more info on Standardization one can look for Wikipedia :)


for img in Voxel_HU_List[:100]:
    # getting Standard Scores for feature sampling
    mean = npy.mean(img)
    std = npy.std(img)
    img = img- mean
    img = img/std      # from formula x'= (x-x_avg)/s
    
print(img[200])

ppt.hist(img.flatten(), bins=50, color='crimson',histtype='bar')
ppt.xlabel("Standardized Values of HU")
ppt.ylabel("Frequency")
ppt.show()

In [ ]:
# Filter operation for distinction of lung area

#from skimage import filters

#edge_img = filters.sobel(Voxel_HU_List[55])
#ppt.imshow(edge_img,cmap=ppt.cm.gray)
#ppt.show()

In [ ]:
#SEGMENTATION using Global Thresholding method

img=Voxel_HU_List[56]
max_ = npy.max(img)
print(max_)
min_ = npy.min(img)
print(min_)
#thres=(max_+min_)/2     #Initial threshold using max and min brightness values

thres=npy.mean(img)

print(thres)
th0=0

array1=img[img<=thres]
print(len(array1))      # First segmented cluster
array2=img[img>thres]
print(len(array2))      # Second segmented cluster
print(array1)
print(array2)

mean1= npy.mean(array1)
mean2= npy.mean(array2)


#while abs(thres-th0)<10^-6:
    #th0=thres
    #thres=(mean1+mean2)/2
    #print (thres)
    #array1=img[img<=thres]
    #array2=img[img>thres]
    
     #mean1= npy.mean(array1)
    #mean2= npy.mean(array2)
    
    
#print(thres)   

thres_img = npy.where(img<thres,1.0,0.0)
ppt.imshow(thres_img,cmap=ppt.cm.gray)
ppt.show()

In [ ]:
import cv2

img=Voxel_HU_List[56]
img = npy.array(img * 255, dtype = npy.uint8)
ret,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)
titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [img, th1, th2, th3]

X=ppt.figure(figsize=(10,10))
for i in range(4):
    X.add_subplot(2,2,i+1),
    ppt.imshow(images[i],'gray')
    ppt.title(titles[i])
    ppt.xticks([]),ppt.yticks([])
ppt.show()

In [ ]:
def plot_3d(image, threshold=-300):
    
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = npy.transpose(image, axes=(2,1,0)) # image.transpose(1,0,2)
    
    verts, faces = measure.marching_cubes(p, threshold)

    fig = ppt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.70)
    face_color = [0.45, 0.45, 0.75]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

 

In [ ]:
##plot_3d(img2, 400)

In [ ]:
def largest_label_volume(im, bg=-1):
    vals, counts = npy.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[npy.argmax(counts)]
    else:
        return None

def segment_lung_mask(image, fill_lung_structures=True):
    
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = npy.array(image > -320, dtype=npy.int8)+1
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    background_label = labels[0,0,0]
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2
    
    
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
            
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1

    
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
    
    # Remove other air pockets insided body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image


In [ ]:


segmented_lungs = segment_lung_mask(img2, False)
segmented_lungs_fill = segment_lung_mask(img2, True)


In [ ]:


#plot_3d(segmented_lungs, 0)


In [ ]:
images= segmented_lungs_fill - segmented_lungs

In [ ]:


MIN_BOUND = -1000.0
MAX_BOUND = 400.0
    
def normalize(images):
    images = (images - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    images[images>1] = 1.
    images[images<0] = 0.
    return images



In [ ]:
PIXEL_MEAN = 0.25

def zero_center(images):
    images = images - PIXEL_MEAN
    return images